In [1]:
import pandas as pd
import pycurl
import io
import re
from bs4 import BeautifulSoup
import datetime
import csv
import json

In [2]:
def readComments(pageNumber):

    c = pycurl.Curl()
    url = 'https://www.eltenedor.es/restaurant/batavia-healthy-food/423897/contentrates?sort=RESERVATION_DATE_DESC&page='+str(pageNumber)+'&restaurantId=423897&filters%5Bwith_comments_only%5D=0&filters%5Bfood_reports_only%5D=0&filters%5Boccasion%5D=ALL'
    print(url)
    buffer = io.BytesIO()
    c.setopt(pycurl.USERAGENT,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36")
    c.setopt(pycurl.COOKIE,"csrftoken=Uz35pSryTMKtdCMtsVqUd6nl8L0Ux8ji; sessionid=qyuswm8l5bc63v6w2pd8yviowjncolva")
    c.setopt(pycurl.URL,url)
    c.setopt(pycurl.FOLLOWLOCATION, True)
    c.setopt(pycurl.HEADER, True)
    c.setopt(pycurl.ACCEPT_ENCODING,'gzip, deflate, br')
    c.setopt(pycurl.WRITEFUNCTION, buffer.write)
    c.setopt(pycurl.HTTPHEADER, ['cache-control: no-cache','accept-language: es-ES,es;q=0.9,en;q=0.8,pl;q=0.7,ru;q=0.6,fr;q=0.5,ca;q=0.4,pt;q=0.3,it;q=0.2','accept:*/*','authority:www.eltenedor.es', 'x-requested-with:XMLHttpRequest'])
    c.setopt(pycurl.REFERER, 'https://www.eltenedor.es/restaurante/batavia-healthy-food/423897')             
    c.perform()
    resp = buffer.getvalue()
    header_len = c.getinfo(pycurl.HEADER_SIZE)
    header = resp[0: header_len]
    body = resp[header_len:]
    c.close()

    return body

def extractRatingsFromReview(review): 

    date = BeautifulSoup(str(review),'html.parser').select('.reviewItem-date')

    if len(date) < 1:
        return None
    
    date = date[0].text
    date = re.search("(\d*)\/(\d*)\/(\d*)", date)
    
    isodate = "%04d-%02d-%02d" % (int(date[3]),int(date[2]),int(date[1]))
    
    ratings = {'date':isodate}
    rating = BeautifulSoup(str(review),'html.parser').select('.rating-ratingValue')[0]
    ratings['rating'] = rating.text.replace(',','.')

    subRatings = BeautifulSoup(str(review),'html.parser').select('.reviewItem-avgRatingWrapper')
    for i in subRatings:
        cat = BeautifulSoup(str(i),'html.parser').select('.reviewItem-avgRatingLabel')[0].text
        value = BeautifulSoup(str(i),'html.parser').select('.reviewItem-scoreText')[0].text
        ratings[cat] = value
    return ratings

def getRatingsFromPage(pageNumber):
    body = readComments(pageNumber)
    content_json = json.loads(body)
    soup = BeautifulSoup(content_json['content'],'html.parser')
    s1 = soup.select(".reviewItem")
    ratings = []
    for i in s1:    
        rr = extractRatingsFromReview(i)
        if rr:
            ratings.append(rr)
    return ratings

In [3]:
allratings = []

for i in range(1,1000):
    ratings = getRatingsFromPage(i)
    if len(ratings) > 0:
        print(ratings)
        allratings = allratings + ratings
    else:
        break
        
print(len(allratings))

https://www.eltenedor.es/restaurant/batavia-healthy-food/423897/contentrates?sort=RESERVATION_DATE_DESC&page=1&restaurantId=423897&filters%5Bwith_comments_only%5D=0&filters%5Bfood_reports_only%5D=0&filters%5Boccasion%5D=ALL
[{'date': '2019-06-04', 'rating': '7', 'Cocina': '8', 'Servicio': '6', 'Ambiente': '6'}, {'date': '2019-05-29', 'rating': '5.5', 'Cocina': '6', 'Servicio': '6', 'Ambiente': '4'}, {'date': '2019-05-25', 'rating': '10', 'Cocina': '10', 'Servicio': '10', 'Ambiente': '10'}, {'date': '2019-05-24', 'rating': '6.5', 'Cocina': '6', 'Servicio': '8', 'Ambiente': '6'}, {'date': '2019-05-22', 'rating': '6', 'Cocina': '6', 'Servicio': '6', 'Ambiente': '6'}, {'date': '2019-05-14', 'rating': '10', 'Cocina': '10', 'Servicio': '10', 'Ambiente': '10'}, {'date': '2019-05-10', 'rating': '10', 'Cocina': '10', 'Servicio': '10', 'Ambiente': '10'}, {'date': '2019-05-04', 'rating': '3.5', 'Cocina': '2', 'Servicio': '2', 'Ambiente': '8'}, {'date': '2019-05-03', 'rating': '8', 'Cocina': '8', 

[{'date': '2018-10-26', 'rating': '8.5', 'Cocina': '8', 'Servicio': '10', 'Ambiente': '8'}, {'date': '2018-10-26', 'rating': '10', 'Cocina': '10', 'Servicio': '10', 'Ambiente': '10'}, {'date': '2018-10-24', 'rating': '10', 'Cocina': '10', 'Servicio': '10', 'Ambiente': '10'}, {'date': '2018-10-24', 'rating': '9', 'Cocina': '10', 'Servicio': '8', 'Ambiente': '8'}, {'date': '2018-10-20', 'rating': '10', 'Cocina': '10', 'Servicio': '10', 'Ambiente': '10'}, {'date': '2018-10-17', 'rating': '10', 'Cocina': '10', 'Servicio': '10', 'Ambiente': '10'}]
https://www.eltenedor.es/restaurant/batavia-healthy-food/423897/contentrates?sort=RESERVATION_DATE_DESC&page=6&restaurantId=423897&filters%5Bwith_comments_only%5D=0&filters%5Bfood_reports_only%5D=0&filters%5Boccasion%5D=ALL
86


In [4]:
js=json.dumps(allratings)

In [5]:
print(js)

[{"date": "2019-06-04", "rating": "7", "Cocina": "8", "Servicio": "6", "Ambiente": "6"}, {"date": "2019-05-29", "rating": "5.5", "Cocina": "6", "Servicio": "6", "Ambiente": "4"}, {"date": "2019-05-25", "rating": "10", "Cocina": "10", "Servicio": "10", "Ambiente": "10"}, {"date": "2019-05-24", "rating": "6.5", "Cocina": "6", "Servicio": "8", "Ambiente": "6"}, {"date": "2019-05-22", "rating": "6", "Cocina": "6", "Servicio": "6", "Ambiente": "6"}, {"date": "2019-05-14", "rating": "10", "Cocina": "10", "Servicio": "10", "Ambiente": "10"}, {"date": "2019-05-10", "rating": "10", "Cocina": "10", "Servicio": "10", "Ambiente": "10"}, {"date": "2019-05-04", "rating": "3.5", "Cocina": "2", "Servicio": "2", "Ambiente": "8"}, {"date": "2019-05-03", "rating": "8", "Cocina": "8", "Servicio": "8", "Ambiente": "8"}, {"date": "2019-04-27", "rating": "9", "Cocina": "10", "Servicio": "10", "Ambiente": "6"}, {"date": "2019-04-27", "rating": "6", "Cocina": "4", "Servicio": "10", "Ambiente": "6"}, {"date": "

In [6]:
df = pd.DataFrame(allratings)
df['rating'] = pd.to_numeric(df['rating'])

In [7]:
df.head()

,Ambiente,Cocina,Servicio,date,rating
0,6,8,6,2019-06-04,7.0
1,4,6,6,2019-05-29,5.5
2,10,10,10,2019-05-25,10.0
3,6,6,8,2019-05-24,6.5
4,6,6,6,2019-05-22,6.0


In [8]:
df.dtypes

Ambiente     object
Cocina       object
Servicio     object
date         object
rating      float64
dtype: object

In [9]:
df.to_csv("/tmp/reviews.csv")

In [10]:
df.groupby('date')['rating'].agg([pd.np.min, pd.np.max, pd.np.mean])

,amin,amax,mean
date,,,
2018-10-17,10.0,10.0,10.000000
2018-10-20,10.0,10.0,10.000000
2018-10-24,9.0,10.0,9.500000
2018-10-26,8.5,10.0,9.250000
2018-10-27,10.0,10.0,10.000000
2018-10-29,10.0,10.0,10.000000
2018-10-31,9.5,10.0,9.750000
2018-11-03,9.5,9.5,9.500000
2018-11-06,9.0,9.0,9.000000


In [46]:
#start = pd.to_datetime(min(df['date']))
start = pd.to_datetime('2018-09-21')
end = pd.to_datetime(max(df['date']))

In [49]:
#start = pd.to_datetime(min(df['date']))
start = pd.to_datetime('2018-09-21')
end = pd.to_datetime(max(df['date']))

rating = float(df.iloc[0].rating)
counter = 1
averages = {}
average = rating

while start < end:
    
    today = start.strftime('%Y-%m-%d')

    if today in df['date'].values:      
        reviewed = float(df[df['date']==today].iloc[0].rating)
        print(reviewed)
        average = ((average * counter) + reviewed)/(counter+1)
        counter += 1

    averages[today] = average
    start = (start + pd.DateOffset(days=1))


10.0
10.0
10.0
8.5
10.0
10.0
9.5
9.5
9.0
10.0
10.0
8.5
9.0
8.5
10.0
10.0
7.5
10.0
8.5
8.5
8.0
8.0
5.0
9.0
6.5
6.0
10.0
9.5
9.5
7.0
9.5
9.0
6.0
10.0
10.0
9.5
10.0
9.0
9.5
10.0
5.5
10.0
8.0
7.5
9.5
10.0
7.5
8.5
10.0
8.0
8.5
9.5
10.0
9.0
8.0
3.5
10.0
10.0
6.0
6.5
10.0
5.5


In [50]:
averages

{'2018-09-21': 7.0,
 '2018-09-22': 7.0,
 '2018-09-23': 7.0,
 '2018-09-24': 7.0,
 '2018-09-25': 7.0,
 '2018-09-26': 7.0,
 '2018-09-27': 7.0,
 '2018-09-28': 7.0,
 '2018-09-29': 7.0,
 '2018-09-30': 7.0,
 '2018-10-01': 7.0,
 '2018-10-02': 7.0,
 '2018-10-03': 7.0,
 '2018-10-04': 7.0,
 '2018-10-05': 7.0,
 '2018-10-06': 7.0,
 '2018-10-07': 7.0,
 '2018-10-08': 7.0,
 '2018-10-09': 7.0,
 '2018-10-10': 7.0,
 '2018-10-11': 7.0,
 '2018-10-12': 7.0,
 '2018-10-13': 7.0,
 '2018-10-14': 7.0,
 '2018-10-15': 7.0,
 '2018-10-16': 7.0,
 '2018-10-17': 8.5,
 '2018-10-18': 8.5,
 '2018-10-19': 8.5,
 '2018-10-20': 9.0,
 '2018-10-21': 9.0,
 '2018-10-22': 9.0,
 '2018-10-23': 9.0,
 '2018-10-24': 9.25,
 '2018-10-25': 9.25,
 '2018-10-26': 9.1,
 '2018-10-27': 9.25,
 '2018-10-28': 9.25,
 '2018-10-29': 9.357142857142858,
 '2018-10-30': 9.357142857142858,
 '2018-10-31': 9.375,
 '2018-11-01': 9.375,
 '2018-11-02': 9.375,
 '2018-11-03': 9.38888888888889,
 '2018-11-04': 9.38888888888889,
 '2018-11-05': 9.38888888888889,
 '2

In [55]:
handle = open("../data/daily_ratings.csv",'wt')
for i in averages:
    handle.write(i + "," + str(averages[i]) + "\n")
handle.close()